In [1]:
import sys
import sys
sys.path.append('/data_nfs/og86asub/netmap/netmap-evaluation/')

from netmap.src.model.negbinautoencoder import *
import scanpy as sc

from sklearn.model_selection import train_test_split
import time
from captum.attr import GradientShap
from netmap.src.model.inferrence_simple import *
from netmap.src.utils.data_utils import attribution_to_anndata
import netmap.src.downstream.downstream as d
adata = sc.read_h5ad("/data_nfs/og86asub/netmap/netmap-evaluation/data/perturb_seq_subsets/CEBPA_KLF1/data.h5ad")
counts = pd.DataFrame(adata.layers['counts'].todense())
counts.columns = adata.var.index

In [2]:
adata = sc.read_h5ad("/data_nfs/og86asub/netmap/netmap-evaluation/data/perturb_seq_subsets/CEBPA_KLF1/data.h5ad")
counts = pd.DataFrame(adata.layers['counts'].todense())
counts.columns = adata.var.index

In [3]:
counts

Gene_IDs,ATAD3C,C1orf233,MMP23B,RP11-345P4.10,PRKCZ,TNFRSF14,SMIM1,LINC01134,ZBTB48,UTS2,...,ITGB2,ITGB2-AS1,DIP2A,MT-ND2,MT-CO2,MT-ATP6,MT-ND4,MT-ND6,AL592183.1,AC240274.1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,19.0,47.0,17.0,28.0,4.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,98.0,203.0,102.0,134.0,4.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,20.0,87.0,34.0,63.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,96.0,181.0,53.0,85.0,13.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,56.0,146.0,47.0,72.0,9.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.0,31.0,21.0,36.0,0.0,0.0,0.0
1434,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,89.0,200.0,91.0,120.0,5.0,0.0,0.0
1435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,67.0,131.0,60.0,83.0,1.0,0.0,0.0
1436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,139.0,279.0,98.0,171.0,5.0,0.0,0.0


In [4]:
import pandas as pd
import numpy as np

def binarize_and_compute_contingency(df_counts):
    """
    Binarizes raw gene count data and computes pairwise contingency tables efficiently.

    Args:
        df_counts (pd.DataFrame): A DataFrame with genes as rows and samples as columns.

    Returns:
        dict: A dictionary containing four DataFrames for the contingency table values:
              'a' (both genes expressed), 'b' (gene A expressed, gene B not),
              'c' (gene A not, gene B expressed), and 'd' (neither expressed).
    """

    # Step 1: Binarize the gene expression data
    # Convert counts to 1 (expressed) or 0 (not expressed)
    df_binary = (df_counts > 0).astype(int)

    # Step 2: Compute pairwise contingency using matrix multiplication
    # 'a': Both genes are expressed (count > 0 in the same sample)
    a_matrix = df_binary.dot(df_binary.T)

    # 'd': Neither gene is expressed (count == 0 in the same sample)
    # We invert the binary matrix to find where both are 0
    df_binary_inverted = 1 - df_binary
    d_matrix = df_binary_inverted.dot(df_binary_inverted.T)

    # Calculate marginal sums for 'b' and 'c'
    # Row-wise sum gives total number of samples where each gene is expressed
    expressed_counts = df_binary.sum(axis=1)

    # 'b': Gene A is expressed, Gene B is not
    # This is calculated as (total samples where Gene A is expressed) - (samples where both are expressed)
    b_matrix = expressed_counts.to_frame().values - a_matrix

    # 'c': Gene B is expressed, Gene A is not
    # This is calculated as (total samples where Gene B is expressed) - (samples where both are expressed)
    # We use a transpose here to align the dimensions correctly
    c_matrix = expressed_counts.to_frame().T.values - a_matrix

    # Correcting for floating point inaccuracies by rounding to nearest integer
    b_matrix = b_matrix.round().astype(int)
    c_matrix = c_matrix.round().astype(int)

    return {
        'a': pd.DataFrame(a_matrix, index=df_counts.index, columns=df_counts.index),
        'b': pd.DataFrame(b_matrix, index=df_counts.index, columns=df_counts.index),
        'c': pd.DataFrame(c_matrix, index=df_counts.index, columns=df_counts.index),
        'd': pd.DataFrame(d_matrix, index=df_counts.index, columns=df_counts.index)
    }






In [5]:
contingency_tables2 = binarize_and_compute_contingency(counts[list(adata.obs['gene_1']=='KLF1')].iloc[np.random.choice(1194, size=244, replace=True)].T)
# Run the function to get the contingency tables
contingency_tables = binarize_and_compute_contingency(counts[list(adata.obs['gene_1']=='CEBPA')].T)

In [56]:
contingency_tables

{'a': Gene_IDs       ATAD3C  C1orf233  MMP23B  RP11-345P4.10  PRKCZ  TNFRSF14  \
 Gene_IDs                                                                  
 ATAD3C              2         1       0              0      0         0   
 C1orf233            1        49       1              0      0         0   
 MMP23B              0         1       9              0      0         0   
 RP11-345P4.10       0         0       0              1      0         0   
 PRKCZ               0         0       0              0      2         0   
 ...               ...       ...     ...            ...    ...       ...   
 MT-ATP6             2        49       9              1      2         7   
 MT-ND4              2        49       9              1      2         7   
 MT-ND6              1        33       7              0      1         3   
 AL592183.1          1         9       0              0      1         0   
 AC240274.1          1         4       2              0      1         2   
 
 Gene

In [ ]:
from scipy.stats import chi2_contingency
import itertools

def compute_stats_mat(contingency_tables):
    loop_end = contingency_tables['a'].shape[1]
    pvalues  = np.array((loop_end, loop_end))
    teststat = np.array((loop_end, loop_end))
    genenames = contingency_tables['a'].columns
    for genea in range(loop_end):
        for geneb in range(genea+1, loop_end):
            contingency_dict = {
                'expressed': {'expressed': contingency_tables['a'].loc[genenames[genea], genenames[geneb]],
                            'not_expressed': contingency_tables['b'].loc[genenames[genea], genenames[geneb]]},
                'not_expressed': {'expressed': contingency_tables['c'].loc[genenames[genea], genenames[geneb]],
                                'not_expressed': contingency_tables['d'].loc[genenames[genea], genenames[geneb]]}
            }
            contingency_df2 = pd.DataFrame(contingency_dict)
            try:
                cont = chi2_contingency(contingency_df2)
                pvalues[genea, geneb] = cont.pvalue
                teststat[genea, geneb] = cont.statistic
            except ValueError:
                pvalues[genea, geneb] = np.nan
                teststat[genea, geneb] = np.nan
    return pvalues, teststat

ATAD3C
C1orf233
ATAD3C
MMP23B
ATAD3C
RP11-345P4.10


ValueError: The internally computed table of expected frequencies has a zero element at (np.int64(0), np.int64(0)).

In [55]:
res

[['ATAD3C', 'C1orf233', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'MMP23B', np.float64(1.0), np.float64(0.0)]]

In [52]:
result_df = pd.DataFrame(res)
result_df.columns = ['GeneA', 'GeneB', 'p_val', 'p_val']

In [53]:
result_df = result_df.loc[:, ['GeneA', 'GeneB', 'p_val']]

In [54]:
result_df.pivot(index='GeneA', columns='GeneB', values='p_val')

ValueError: Data must be 1-dimensional, got ndarray of shape (2, 2) instead

In [32]:
result_df[result_df.statistic>0.001][result_df.GeneA == 'ZNRF3-AS1']

,GeneA,GeneB,statistic
1997290,ZNRF3-AS1,PATZ1,0.098734
1997297,ZNRF3-AS1,RAC2,4.593324
1997298,ZNRF3-AS1,SH3BP1,0.024935
1997299,ZNRF3-AS1,LGALS1,0.281588
1997305,ZNRF3-AS1,TNRC6B,0.003088
1997306,ZNRF3-AS1,MKL1,0.002306
1997310,ZNRF3-AS1,BIK,11.502371
1997311,ZNRF3-AS1,KIAA0930,0.017324
1997314,ZNRF3-AS1,TTC38,0.072435
1997316,ZNRF3-AS1,LMF2,0.378443


In [50]:
# Example of a full contingency table for a specific pair (e.g., GeneA vs GeneB)
geneA = 'AAK1'
geneB = 'ZNRF3-AS1'
contingency_dict = {
    'expressed': {'expressed': contingency_tables2['a'].loc[geneA, geneB],
                'not_expressed': contingency_tables2['b'].loc[geneA, geneB]},
    'not_expressed': {'expressed': contingency_tables2['c'].loc[geneA, geneB],
                    'not_expressed': contingency_tables2['d'].loc[geneA, geneB]}
}
cont = chi2_contingency(contingency_df2)
cont


ValueError: The internally computed table of expected frequencies has a zero element at (np.int64(0), np.int64(0)).

In [41]:
res

[['ATAD3C', 'C1orf233', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'MMP23B', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'RP11-345P4.10', nan, nan],
 ['ATAD3C', 'PRKCZ', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'TNFRSF14', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'SMIM1', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'LINC01134', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C',
  'ZBTB48',
  np.float64(0.04624471067210825),
  np.float64(3.972658343028713)],
 ['ATAD3C', 'UTS2', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'GPR157', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'H6PD', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'PIK3CD', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C',
  'PGD',
  np.float64(0.604651236290447),
  np.float64(0.2680375680791826)],
 ['ATAD3C', 'MTHFR', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'TNFRSF1B', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'EFHD2', np.float64(1.0), np.float64(0.0)],
 ['ATAD3C', 'FLJ37453', np.float64(1

In [8]:
from statsmodels.stats.contingency_tables import StratifiedTable

def perform_cmh_test(table1, table2):
    """
    Performs the Cochran-Mantel-Haenszel (CMH) test on two 2x2 contingency tables.

    Args:
        table1 (pd.DataFrame or np.array): Contingency table for the first subgroup.
        table2 (pd.DataFrame or np.array): Contingency table for the second subgroup.

    Returns:
        dict: A dictionary containing the CMH test statistic, p-value, and degrees of freedom.
    """
    # Combine the tables into a 3D NumPy array
    # The structure should be (rows, columns, number_of_subgroups)

    # Create a StratifiedTable object.
    stratified_table = StratifiedTable([table1, table2])

    # Perform the CMH test for nominal association
    cmh_result = stratified_table.test_equal_odds()

    # Access the CMH test results from the results object
    return cmh_result




In [9]:
lr = perform_cmh_test(contingency_df, contingency_df2)

In [10]:
print(lr)

pvalue      0.09860722836677871
statistic   2.7279265491900344


In [11]:

# Example of a full contingency table for a specific pair (e.g., GeneA vs GeneB)
geneB = 'PGD'
geneA = 'KLF1'
contingency_dict = {
    'expressed': {'expressed': contingency_tables['a'].loc[geneA, geneB],
                  'not_expressed': contingency_tables['b'].loc[geneA, geneB]},
    'not_expressed': {'expressed': contingency_tables['c'].loc[geneA, geneB],
                      'not_expressed': contingency_tables['d'].loc[geneA, geneB]}
}
contingency_df = pd.DataFrame(contingency_dict)
contingency_df.index.name = geneB
contingency_df.columns.name = geneA
print(f"Full Contingency Table for {geneA} vs {geneB}:")
print(contingency_df)
chi2_contingency(contingency_df)

Full Contingency Table for KLF1 vs PGD:
KLF1           expressed  not_expressed
PGD                                    
expressed             48            157
not_expressed         16             23


Chi2ContingencyResult(statistic=np.float64(4.381278055729275), pvalue=np.float64(0.03633574234913739), dof=1, expected_freq=array([[ 53.7704918, 151.2295082],
       [ 10.2295082,  28.7704918]]))

In [12]:


# Example of a full contingency table for a specific pair (e.g., GeneA vs GeneB)
geneB = 'PGD'
geneA = 'KLF1'
contingency_dict = {
    'expressed': {'expressed': contingency_tables2['a'].loc[geneA, geneB],
                  'not_expressed': contingency_tables2['b'].loc[geneA, geneB]},
    'not_expressed': {'expressed': contingency_tables2['c'].loc[geneA, geneB],
                      'not_expressed': contingency_tables2['d'].loc[geneA, geneB]}
}
contingency_df2 = pd.DataFrame(contingency_dict)
contingency_df2.index.name = geneB
contingency_df2.columns.name = geneA
print(f"Full Contingency Table for {geneA} vs {geneB}:")
print(contingency_df2)
chi2_contingency(contingency_df2)

Full Contingency Table for KLF1 vs PGD:
KLF1           expressed  not_expressed
PGD                                    
expressed            160             18
not_expressed         51             15


Chi2ContingencyResult(statistic=np.float64(5.5170549562254045), pvalue=np.float64(0.018831936740757244), dof=1, expected_freq=array([[153.92622951,  24.07377049],
       [ 57.07377049,   8.92622951]]))

In [13]:
# Run the CMH test


In [14]:
np.sum(datax['total'])

NameError: name 'datax' is not defined

In [ ]:
import pandas as pd
from statsmodels.regression.linear_model import RegressionResults
from statsmodels.genmod import families
from scipy.stats import chi2
import statsmodels.formula.api as form
import statsmodels.api as sm
# 1. Create the DataFrame
datax = pd.DataFrame({
    'sample': [1, 1, 2, 2],
    'time1': [1, 2, 1, 2],
    'pos': [25, 45, 15, 30],
    'neg': [5, 20, 25, 30]
})
datax['total'] = datax['pos'] + datax['neg']

n = datax['total'].sum()

X = sm.add_constant(datax.loc[:, ['sample', 'time1']])
# 3. Create the GLM object, passing the data and the family
model = sm.GLM(datax['pos'],X , family=sm.families.Binomial())


# 4. Call the 'fit()' method on the GLM model object
results  = model.fit_regularized()


In [ ]:
results.summary()

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Create a sample DataFrame
data_exam = pd.DataFrame({
  'hours_studied': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],
  'passed': [0, 0, 0, 1, 0, 1, 1, 1, 1, 1] # 0 = Failed, 1 = Passed
})

# Define the response (y) and predictor (X) variables
y = data_exam['passed']
X = data_exam[['hours_studied']]

# Add a constant (intercept) to the predictor variables
X = sm.add_constant(X)

# Run logistic regression
# The Logit class automatically uses a logit link function
model = sm.Logit(y, X).fit()

# Print a summary of the model results
print(model.summary())

# Make a prediction
# Predict the probability of passing for someone who studied for 2.2 hours
new_data = pd.DataFrame({'const': [1], 'hours_studied': [2.2]})
prediction = model.predict(new_data)
print(f"\nPredicted probability of passing: {prediction.values[0]:.4f}")

In [ ]:
exog

In [ ]:

# 4. Fit the second model with the interaction term
exog_interaction = sm.add_constant(datax[['sample', 'time1']])
exog_interaction['sample_time1'] = datax['sample'] * datax['time1']

m2 = sm.Logit(endog, exog_interaction)
m2 = m2.fit()
print(m2.summary())

# 5. Perform the Likelihood Ratio Test (LRT)
ll_m1 = m1.llf
ll_m2 = m2.llf
lrt_statistic = -2 * (ll_m1 - ll_m2)
df = m2.df_model - m1.df_model
p_value = chi2.sf(lrt_statistic, df)

print("\nLikelihood Ratio Test (LRT):")
print(f"LRT Statistic: {lrt_statistic:.4f}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value:.4f}")

In [ ]:
logit_res.llf

In [ ]:
import pandas as pd
datax = pd.DataFrame({
    'sample': [1, 1, 2, 2],
    'time1': [1, 2, 1, 2],
    'pos': [25, 45, 15, 30],
    'neg': [5, 20, 25, 30]
})
datax['total'] = datax['pos'] + datax['neg']
datax['frac'] = datax['pos'] / datax['total']

# Check for complete separation by 'sample'
print(datax.groupby('sample')['frac'].agg(['min', 'max']))

# Check for complete separation by 'time1'
print(datax.groupby('time1')['frac'].agg(['min', 'max']))

In [ ]:
m1.summary()

In [ ]:
m2.summary()

In [ ]:
datax

In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
corr = np.corrcoef(data[data.obs['gene_1'] == 'CEBPA', :].X[:, 400:600])
corr2 = spearmanr(data[data.obs['gene_1'] == 'KLF1', :].X[:, 400:600]).correlation

corr[pd.isna(corr)] = 0
corr2[pd.isna(corr2)] = 0
# Ensure the correlation matrix is symmetric
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1)

corr2 = (corr2 + corr2.T) / 2
np.fill_diagonal(corr2, 1)

#corr = corr-corr2
np.fill_diagonal(corr, 1)
# We convert the correlation matrix to a distance matrix before performing
# hierarchical clustering using Ward's linkage.
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
dendro = hierarchy.dendrogram(
    dist_linkage, labels=data.var.index.to_list()[400:600], ax=ax1, leaf_rotation=90
)
dendro_idx = np.arange(0, len(dendro["ivl"]))

ax2.imshow(corr[dendro["leaves"], :][:, dendro["leaves"]])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro["ivl"], rotation="vertical")
ax2.set_yticklabels(dendro["ivl"])
_ = fig.tight_layout()


In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
corr = spearmanr(data[data.obs['gene_1'] == 'KLF1', :].X[:, 500:750]).correlation
corr[pd.isna(corr)] = 0

# Ensure the correlation matrix is symmetric
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1)

# We convert the correlation matrix to a distance matrix before performing
# hierarchical clustering using Ward's linkage.
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
dendro = hierarchy.dendrogram(
    dist_linkage, labels=data.var.index.to_list()[500:750], ax=ax1, leaf_rotation=90
)
dendro_idx = np.arange(0, len(dendro["ivl"]))

ax2.imshow(corr[dendro["leaves"], :][:, dendro["leaves"]])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro["ivl"], rotation="vertical")
ax2.set_yticklabels(dendro["ivl"])
_ = fig.tight_layout()


In [ ]:
print(f"case: {case}")
print("\n")

case_ = os.path.basename(os.path.normpath(case))
case_ = case_.split("_")
#net_gt_1 = pd.read_csv(f'{case}{case_[0]}_network_wilcoxon_no_pb.csv')
#net_gt_2 = pd.read_csv(f'{case}{case_[1]}_network_wilcoxon_no_pb.csv')
#print(f"Size GT NET 1 : {net_gt_1.shape}")
#print(f"Size GT NET 1 : {net_gt_2.shape}")

In [ ]:
import sys
sys.path.append('/data_nfs/og86asub/netmap/netmap-evaluation/netmap')

from src.utils.netmap_config import NetmapConfig

In [ ]:
NetmapConfig.learning_rate

In [ ]:

config = NetmapConfig.read_yaml(yaml_file="/data_nfs/og86asub/netmap/netmap-evaluation/results/configurations/netmap/")
net = pd.read_csv('/data_nfs/og86asub/netmap/NetMap_LRP/benchmark/data/simulated_data/config_ro/net_63_4684/net.tsv', sep = '\t')


In [ ]:
## load data
#eval_config = read_config('/home/baz8031/NetMap/2025/4/NetMap_LRP/evaluation/configurations/netmap/eval_config.yaml')

collectri = pd.read_csv('benchmark/data/input_network/collectri_2024-11-11.tsv', sep='\t')
net_1 = pd.read_csv(f'{case}{case_[0]}_network_DE_DESeq2_and_NO_pb.csv')
net_2 = pd.read_csv(f'{case}{case_[1]}_network_DE_DESeq2_and_NO_pb.csv')


#collectri = pd.read_csv('benchmark/data/input_network/collectri_2024-11-11.tsv', sep='\t')
#nn = pd.read_csv(config['evaluation']['network_1'])
#nn2 = pd.read_csv(config['evaluation']['network_2'])

#result_dict = {'gene_1': config['evaluation']['gene_1']}
#result_dict['gene_2'] =  config['evaluation']['gene_2']


## load data
adata_full = sc.read_h5ad('/home/baz8031/NetMap/perturb-seq/Now/perterb-seq-main-raw.h5ad')

In [ ]:
adata = adata_full[adata_full.obs.guide_identity.isin([f'{case_[0]}_NegCtrl0', f'{case_[1]}_NegCtrl0'])].copy()
adata.obs['guide_identity'][adata.obs['guide_identity']!=f'{case_[1]}_NegCtrl0']= f'{case_[0]}_NegCtrl0'

adata.layers['counts'] = adata.X
#sc.pp.normalize_total(adata)
#adata.raw = adata

In [ ]:
adata.var_names_make_unique()
adata = adata[adata.obs.good_coverage].copy()
adata = adata[adata.obs.number_of_cells == 1]
sc.pp.normalize_total(adata)
sc.pp.filter_genes_dispersion(adata, n_top_genes=2000)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata, resolution=0.1)
sc.tl.umap(adata)

In [ ]:
gene_names = np.array(adata.var.index)

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata, resolution=0.1)
sc.tl.umap(adata)
sc.pl.umap(adata, color = ['leiden', 'guide_identity'])

In [ ]:
import pandas as pd

sc.tl.rank_genes_groups(adata, groupby='guide_identity', method='wilcoxon') 
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names

# Collect top N DEGs for each group
top_n = 100
degs = {
    group: pd.DataFrame({
        'names': result['names'][group][:top_n],
        'logfoldchanges': result['logfoldchanges'][group][:top_n],
        'pvals_adj': result['pvals_adj'][group][:top_n]
    })
    for group in groups
}

gt_nets = []
for k in degs.keys():
    print(k.split("_")[0])
    print(list(degs[k]["names"]))

    edges = [(k.split("_")[0], gene) for gene in list(degs[k]["names"])]
    network_df = pd.DataFrame(edges, columns=["source", "target"])
    gt_nets.append(network_df)




In [ ]:
net_gt_1 = gt_nets[0]
net_gt_1.head()

In [ ]:
net_gt_2 = gt_nets[1]
net_gt_2.head()

In [ ]:
adata

In [ ]:
def create_model_zoo(data_tensor, n_models = 4):
    model_zoo = []
    for _ in range(n_models):


        start = time.monotonic()
        data_train2, data_test2 = train_test_split(data_tensor,test_size=0.95, shuffle=True)

        trained_model2 = NegativeBinomialAutoencoder(input_dim=data_tensor.shape[1], latent_dim=10, dropout_rate = 0.25)
        trained_model2 = trained_model2.cuda()

        optimizer2 = torch.optim.Adam(trained_model2.parameters(), lr=1e-4)

        trained_model2 = train_autoencoder(
                trained_model2,
                data_train2.cuda(),
                optimizer2,
                num_epochs=3000

            )
        model_zoo.append(trained_model2)
    return model_zoo



In [ ]:

from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import HDBSCAN


def set_latent_true(model_zoo):
    for mo in model_zoo:
        mo.forward_mu_only = False
        mo.forward_theta_only = False
        mo.latent_only = True
    return model_zoo


def set_all_false(model_zoo):
    for mo in model_zoo:
        mo.forward_mu_only = False
        mo.forward_theta_only = False
        mo.latent_only = False
    return model_zoo


def pair_cooccurence(u, v):
  u_arr = np.array(u)
  v_arr = np.array(v)
  term1 = np.sum(u_arr-v_arr!=0)
  return term1 / len(u_arr)


In [ ]:
def compute_consensus_clustering(model_zoo, data_tensor):
    model_zoo = set_latent_true(model_zoo)
    clusterings = {}
    for m in range(len(model_zoo)):
        latent = model_zoo[m](data_tensor.cuda())
        latent = sc.AnnData(latent.detach().cpu().numpy())
        sc.pp.neighbors(latent)
        sc.tl.umap(latent)
        latent.obs = adata.obs
        sc.tl.leiden(latent, resolution=0.1)
        clusterings[f'leiden_{m}'] = latent.obs[f'leiden']
        sc.pl.umap(latent, color =['guide_identity', "leiden"])
    model_zoo = set_all_false(model_zoo)

    clusterings = pd.DataFrame(clusterings)

    latent.obs = pd.concat([latent.obs, clusterings], axis=1)
    clustering = clusterings.values.astype(int) 

    pairwise_distances_flat = pdist(clustering, metric=pair_cooccurence)
    pairwise_distances_matrix = squareform(pairwise_distances_flat)

    hdb = HDBSCAN(min_cluster_size=50, metric= 'precomputed')
    hdb.fit(pairwise_distances_matrix)
    latent.obs['consensus'] = hdb.labels_


    return latent

In [1]:
def set_mu_true(model_zoo):
    # forward only mu
    for mo in model_zoo:
        mo.forward_mu_only = True
        mo.forward_theta_only = False
        mo.latent_only = False
        mod.forward_pi_only = False
    return model_zoo

In [ ]:
def generate_background_data(model_zoo, data_tensor, latent):
    backgrounds = {}
    for i in latent.obs['consensus'].unique():
        if i!=-1:
            backgrounds[i] = []

    for m in range(len(model_zoo)):
        # Get average params over all 
        mean_theta = get_thetas(model_zoo[m], data_tensor)
        mean_mu = get_mus(model_zoo[m], data_tensor)
        samples_mean = sample_from_vectors(mean_mu, mean_theta, size=200).T


        mean_mus = get_mus_grouping(model_zoo[m], data_tensor, latent.obs['consensus'])
        mean_thetas = get_thetas_grouping(model_zoo[m], data_tensor, latent.obs['consensus'])

        for i in backgrounds.keys():
            backgrounds[i].append(sample_from_vectors(mean_mus[i], mean_thetas[i], size=200).T)

    for b in backgrounds.keys():
        backgrounds[b] = np.concatenate(backgrounds[b])
        backgrounds[b] = torch.tensor(backgrounds[b]).cuda()

    return backgrounds


In [ ]:

def compute_attributions(model_zoo, data_tensor, latent, gene_names, backgrounds, n_top = 250):
    myexplainers = [GradientShap(mo) for mo in model_zoo]
    attributions = []
    for b in backgrounds.keys():
        aggregated_attribution, nlc = inference_complete(myexplainers, data_tensor[latent.obs['consensus'] == b].cuda(), gene_names, backgrounds[b], xai_type='shap-like', num_iterations=5,n_top_genes=n_top )

        cou = pd.DataFrame(nlc)
        cou['target'] = list(np.repeat(adata.var.index, n_top))
        cou.columns = ['source', 'target']

        ad = attribution_to_anndata(aggregated_attribution, varnames=cou, obs=latent.obs[latent.obs['consensus'] == b])
        ad.var = cou
        attributions.append(ad)
    return attributions
    
    

In [ ]:
def concatenate_cluster_anndatas(attributions):
    for a in attributions:
        a.var_names = [f'{z[0]}_{z[1]}' for z in zip(a.var['source'], a.var['target'])]
        a.var_names = [f'{z[0]}_{z[1]}' for z in zip(a.var['source'], a.var['target'])]
    aa = sc.concat(attributions, join = 'outer')
    return aa

In [ ]:
data_tensor = torch.tensor(adata.layers['counts'].todense(), dtype=torch.float32)

model_zoo = create_model_zoo(data_tensor, n_models=2)
latent = compute_consensus_clustering(model_zoo, data_tensor)
backgrounds = generate_background_data(model_zoo, data_tensor, latent)
attributions = compute_attributions(model_zoo, data_tensor, latent, gene_names, backgrounds)

In [ ]:
attributions

In [ ]:
aa = concatenate_cluster_anndatas(attributions)
aa

In [ ]:
aa.var["source"] = aa.var.index.str.split("_").str[0]
aa.var["target"] = aa.var.index.str.split("_").str[1]
aa.var.head(4)

In [ ]:
print(f" Intersection with GT1 I: {pd.merge(aa.var, net_gt_1, on=['source', 'target'], how='inner').shape}")
print(f" Intersection with GT2 I: {pd.merge(aa.var, net_gt_2, on=['source', 'target'], how='inner').shape}")

## Downstream Main

In [ ]:
import postprocessing.downstreammain as d

In [ ]:
# Replace NaN values with 0 if needed

if isinstance(aa.X, np.ndarray):
    aa.X = np.nan_to_num(aa.X)
else:
    aa.X.data[np.isnan(aa.X.data)] = 0

In [ ]:
help(d.downstream_recipe)

In [ ]:
aa = d.downstream_recipe(aa, min_perc=10, n_clusters=2)
aa

In [ ]:
sc.pl.umap(aa, color = ['consensus', "leiden",'guide_identity'], wspace=0.1)

In [ ]:
print(f" Intersection with GT1 I: {pd.merge(aa.var, net_gt_1, on=['source', 'target'], how='inner').shape}")
print(f" Intersection with GT2 I: {pd.merge(aa.var, net_gt_2, on=['source', 'target'], how='inner').shape}")

In [ ]:
help(d.filter_low_count_edges)

In [ ]:
aa = d.filter_low_count_edges(aa, min_mean_count=0.005)
aa

In [ ]:
print(f" Intersection with GT1 I: {pd.merge(aa.var, net_gt_1, on=['source', 'target'], how='inner').shape}")
print(f" Intersection with GT2 I: {pd.merge(aa.var, net_gt_2, on=['source', 'target'], how='inner').shape}")

In [ ]:
gene_inter_adata = aa.copy()
print("Gene interaintion adata")
print(gene_inter_adata)
print("\n\n Gene expression adata")
adata = adata[gene_inter_adata.obs_names,:]
adata

In [ ]:
sc.pl.umap(gene_inter_adata, color = ['consensus', 'leiden', 'guide_identity'])  

In [ ]:
help(d.filter_signatures_by_cluster)

In [ ]:
results_df, all_edges, data = d.filter_signatures_by_cluster(adata, gene_inter_adata, cluster="1", top_per_source=250, min_regulon_size=10, ncores=100)

In [ ]:
help(d.filter_grn_by_top_signatures)

In [ ]:
grn_adata_ = d.filter_grn_by_top_signatures(gene_inter_adata, results_df, all_edges, keep_top_ranked=100)
grn_adata_

In [ ]:
print(f"grn_adata_ : {grn_adata_}")

print(f" Intersection with GT1 Adjusted_P_Value: {pd.merge(grn_adata_.var, net_gt_1, on=['source', 'target'], how='inner').shape}")
print(f" Intersection with GT2 Adjusted_P_Value: {pd.merge(grn_adata_.var, net_gt_2, on=['source', 'target'], how='inner').shape}")
print("\n")

In [ ]:
help(d.create_regulon_activity_adata) 

In [ ]:
adata_regl = d.create_regulon_activity_adata(data, gene_inter_adata, grn_adata_)
adata_regl

In [ ]:
marker_genes = list(adata_regl.var.index)

ax = sc.pl.matrixplot(adata_regl, marker_genes, groupby='leiden', cmap='bwr')

In [ ]:
sc.pl.umap(adata_regl, color = ['consensus', 'guide_identity',  "CEBPE_regulon"], wspace=0.3, cmap='RdBu_r')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations

# Load your data
df = grn_adata_.var

# Step 1: Build mapping from source to set of targets
source_targets = df.groupby('source')['target'].apply(set)

# Step 2: Create similarity matrix
sources = source_targets.index.tolist()
shared_target_matrix = pd.DataFrame(0, index=sources, columns=sources)

# Step 3: Count shared targets for each source pair
for src1, src2 in combinations(sources, 2):
    shared = len(source_targets[src1] & source_targets[src2])
    shared_target_matrix.loc[src1, src2] = shared
    shared_target_matrix.loc[src2, src1] = shared

# Fill diagonal with number of targets per source
for src in sources:
    shared_target_matrix.loc[src, src] = len(source_targets[src])

# Step 4: Clustered heatmap
sns.clustermap(shared_target_matrix, cmap='RdBu_r', figsize=(6, 6), metric='euclidean', method='average')
plt.suptitle('Clustered Heatmap of Shared Targets Between Sources', y=1.05)
plt.show()

In [ ]:
tf_of_intrest = "CEBPE"

net_df = grn_adata_.var[grn_adata_.var["source"] == tf_of_intrest]
net_df.head(4)

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

df = net_df
G = nx.Graph()

for _, row in df.iterrows():
    G.add_edge(row['source'], row['target'])

pos = nx.spring_layout(G, seed=42)
source_nodes = df['source'].unique()

plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(G, pos, nodelist=[node for node in G.nodes if node not in source_nodes], node_size=50, node_color="blue", alpha=0.5)
nx.draw_networkx_nodes(G, pos, nodelist=source_nodes, node_size=1000, node_color="orange", alpha=1, node_shape='^')
nx.draw_networkx_edges(G, pos, edge_color="gray", alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=11, font_weight="bold")

plt.title('')
plt.axis('off')
plt.show()

In [ ]:
help(d.curate_network)

In [ ]:
net_df_curated = d.curate_network(grn_adata_, organism="human", tfs=[tf_of_intrest])

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Data
df = net_df_curated

# Create graph
G = nx.DiGraph()
for _, row in df.iterrows():
    G.add_edge(row['source'], row['target'], type=row['mode'])

# Colors for edges
colors = {'Activation': 'green', 'Repression': 'red', 'Unknown': 'gray'}
edge_colors = [colors[G[u][v]['type']] for u, v in G.edges()]

# Positions
pos = nx.spring_layout(G, seed=42)

# Nodes
tf_nodes = set(df['source'])
nx.draw_networkx_nodes(G, pos, nodelist=tf_nodes, node_color='orange', node_shape='^')
nx.draw_networkx_nodes(G, pos, nodelist=set(G.nodes) - tf_nodes, node_color='lightblue')
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos, edge_color=edge_colors, arrows=True)

# Legend
plt.legend(handles=[
    Line2D([0], [0], color=c, lw=2, label=t) for t, c in colors.items()
] + [
    Line2D([0], [0], marker='^', color='w', label='TF', markerfacecolor='orange', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='TG', markerfacecolor='lightblue', markersize=10)
], title='Legend')
plt.title("Gene Regulation Network")
plt.axis('off')
plt.show()

In [ ]:
# All
net_df_curated = d.curate_network(grn_adata_, organism="human")

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Data
df = net_df_curated

# Create graph
G = nx.DiGraph()
for _, row in df.iterrows():
    G.add_edge(row['source'], row['target'], type=row['mode'])

# Colors for edges
colors = {'Activation': 'green', 'Repression': 'red', 'Unknown': 'gray'}
edge_colors = [colors[G[u][v]['type']] for u, v in G.edges()]

# Positions
pos = nx.spring_layout(G, seed=42)

# Nodes
tf_nodes = set(df['source'])
nx.draw_networkx_nodes(G, pos, nodelist=tf_nodes, node_color='orange', node_shape='^')
nx.draw_networkx_nodes(G, pos, nodelist=set(G.nodes) - tf_nodes, node_color='lightblue')
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos, edge_color=edge_colors, arrows=True)

# Legend
plt.legend(handles=[
    Line2D([0], [0], color=c, lw=2, label=t) for t, c in colors.items()
] + [
    Line2D([0], [0], marker='^', color='w', label='TF', markerfacecolor='orange', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='TG', markerfacecolor='lightblue', markersize=10)
], title='Legend')
plt.title("Gene Regulation Network")
plt.axis('off')
plt.show()

## Run

In [ ]:
aa = concatenate_cluster_anndatas(attributions)
aa.var["source"] = aa.var.index.str.split("_").str[0]
aa.var["target"] = aa.var.index.str.split("_").str[1]

In [ ]:
# Replace NaN values with 0 if needed

if isinstance(aa.X, np.ndarray):
    aa.X = np.nan_to_num(aa.X)
else:
    aa.X.data[np.isnan(aa.X.data)] = 0

In [ ]:
# Downstream processing defaults
DEFAULT_MIN_PERC = 10           # Minimum % of cells expressing a gene to keep it
DEFAULT_N_CLUSTERS = 2          # Number of clusters for filtering/labeling

# Edge filtering
DEFAULT_MIN_MEAN_COUNT = 0.005  # Minimum mean edge weight (strength) to retain

# Signature filtering
DEFAULT_TOP_PER_SOURCE = 200    # Max signatures to retain per source gene
DEFAULT_MIN_REGULON_SIZE = 10   # Minimum number of targets in a regulon to retain it
DEFAULT_NCORES = 100            # Number of parallel workers to use

# GRN filtering
DEFAULT_KEEP_TOP_RANKED = 100    # Top N signatures to keep from filtered results


In [ ]:
# Apply defaults explicitly for clarity
aa = d.downstream_recipe(aa, min_perc=DEFAULT_MIN_PERC, n_clusters=DEFAULT_N_CLUSTERS)

aa = d.filter_low_count_edges(aa, min_mean_count=DEFAULT_MIN_MEAN_COUNT)

gene_inter_adata = aa.copy()

# Align adata to gene interaction AnnData
adata = adata[gene_inter_adata.obs_names, :]

clusters_adata = []
for cluster_ in gene_inter_adata.obs["leiden"].unique():

    print(f"Cluster: {cluster_}")
    # Filter significant signatures
    results_df, all_edges, data = d.filter_signatures_by_cluster(
        adata,
        gene_inter_adata,
        cluster=cluster_,
        top_per_source=DEFAULT_TOP_PER_SOURCE,
        min_regulon_size=DEFAULT_MIN_REGULON_SIZE,
        ncores=DEFAULT_NCORES
    )

    # Keep only most significant interactions
    grn_adata_ = d.filter_grn_by_top_signatures(
        gene_inter_adata,
        results_df,
        all_edges,
        keep_top_ranked=DEFAULT_KEEP_TOP_RANKED
    )
    clusters_adata.append(grn_adata_)
    print(f"Cluster {cluster_} GRN size: {grn_adata_.shape}")



In [ ]:
from anndata import concat

merged_grn_adata = concat(clusters_adata, axis=1)
merged_grn_adata

In [ ]:
print(f" Intersection with GT1 Adjusted_P_Value: {pd.merge(merged_grn_adata.var, net_gt_1, on=['source', 'target'], how='inner').shape}")
print(f" Intersection with GT2 Adjusted_P_Value: {pd.merge(merged_grn_adata.var, net_gt_2, on=['source', 'target'], how='inner').shape}")
print("\n")